In [2]:
!pip install groq
!pip install pinecone-client transformers
!pip install "pinecone-client[grpc]"
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer,  AutoModelForCausalLM

2024-08-06 09:53:59.932383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 09:53:59.932524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 09:54:00.116864: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [8]:
# Initialize Pinecone
pc = Pinecone(api_key='64396515-041d-4b41-adbc-146b4c648311')

# Access the index
index = pc.Index("ai-doctor")

# Load the tokenizer and model for embeddings
# tokenizer_emb = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model_emb = SentenceTransformer("neuml/pubmedbert-base-embeddings")

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [42]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2',max_length=512,device='cpu')
def scores(query,docs):
    response = [[query,doc_text] for doc_text in docs ]
    score = cross_encoder.predict(response)
    return score

In [61]:
import base64
import zlib

def decode_and_decompress_metadata(encoded_metadata):
    # Decode the Base64 string to compressed bytes
    compressed_metadata = base64.b64decode(encoded_metadata)
    # Decompress the bytes
    metadata = zlib.decompress(compressed_metadata).decode('utf-8')
    return metadata

def Retrieve(text, model=model_emb, index=index, top_k=1):
    # Generate the embedding for the input text
    embedding = model_emb.encode(text).tolist()

    # Retrieve the top-k results from Pinecone
    query_response = index.query(vector=embedding, top_k=25, include_metadata=True, include_values=True)

    # Decode and decompress metadata for each result
    results = []
    for match in query_response['matches']:
        results.append(decode_and_decompress_metadata(match['metadata']['patient_doctor_dialogue']))

        
    return results

In [90]:
def generate_response_with_groq(query):
        context=Retrieve(query)
        index = [i for i in range(len(context))]
        score = scores(query,context)
        res = list(zip(index,score,context))
        sorted_res = sorted(res, key=lambda x: x[1],reverse = True)
        context = [ x[2] for x in sorted_res ]
        context = context[0:5]
        context = " \n".join(context)
#         print("\n\n Cotext: ",context)
        prompt_template = """You are a doctor bot. Provide a diagnosis based on the user's mentioned symptoms.

        Question: {query}

        Context: {context}

        Instruction:
        1. Stay strictly within the provided context, query, and relevant medical information. 
        2. If unsure, respond with "I don't know."
        3. The answer should be relevant, comprehensive.
        4. Response should not exceed 2-3 sentences.

        Answer: Based on the provided information, the possible diagnosis is:"""
        prompt = prompt_template.format(context=context, query=query)
        messages=[

                    {"role": "user", "content": prompt}

            ]
        chat_completion = client.chat.completions.create(
            messages = messages,
            model="llama3-8b-8192",
            temperature=0,
            max_tokens=1000
        )
        return chat_completion.choices[0].message.content

In [89]:
#testing
generate_response_with_groq(
    "I feel head ache when i do some physical work"
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Based on the symptoms mentioned, the possible diagnosis is a combination of stress-related conditions, including tension-type headache, neck strain, and stomach ache. The symptoms of headache, stiff and sore body, and abdominal pain are likely related to stress and tension, which can be exacerbated by lack of exercise and adequate hydration.'

In [13]:
#Irrelevant question
generate_response_with_groq(
    "what is astrology."
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"I'm a doctor bot, and I don't provide information on non-medical topics. Astrology is not a medical concept, and I don't have any information about it. I can only provide diagnoses and advice based on medical and physiological contexts. If you have any symptoms or concerns, please feel free to share them, and I'll do my best to help you."

In [14]:
#Few symptoms mentioned
generate_response_with_groq(
    "I feel cold and have nausea."
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Based on your symptoms of feeling cold and nausea, I would like to consider a few possibilities. Your description of feeling cold and having nausea is similar to what I've seen in cases of hypothermia, which is a condition where the body's temperature drops below normal. However, your reported temperature of 94.7°F (35.4°C) is actually slightly low, but not alarmingly so.\n\nAnother possibility could be a viral infection, such as the flu or a stomach bug, which can cause nausea, fatigue, and a general feeling of being unwell. The fact that you felt fine after vomiting and then fell asleep could be indicative of a viral infection.\n\nI would recommend that you consult a doctor to rule out any underlying medical conditions that could be causing your symptoms. They may perform some tests, such as a blood test or a urinalysis, to determine the cause of your symptoms.\n\nIn the meantime, I would suggest that you try to stay hydrated by drinking plenty of fluids, such as water or clear brot

In [15]:
#Typos
generate_response_with_groq(
    "whap do i doooo. i feel like i faintp"
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"I'm concerned about your symptoms. You mentioned feeling like you're going to faint, and you've had episodes where you've fainted, including one incident where you hit your head and left side of your body on the tile. You're also experiencing a burning sensation on the side of your head and arm.\n\nBased on your symptoms, I'm going to take a closer look at a few possibilities. It's possible that you're experiencing vasovagal syncope, which is a common cause of fainting. This occurs when the nerves that control heart rate and blood pressure are stimulated, causing a sudden drop in blood pressure and a decrease in blood flow to the brain.\n\nHowever, given your history of fainting and the recent incident where you hit your head, I'm also concerned that you may have experienced a more serious episode of syncope, such as a seizure or a cardiac arrhythmia.\n\nI would recommend that you schedule an appointment with your primary care physician to discuss your symptoms further. They may want 